## Collect experimental data and process csv

In [ ]:
from pathlib import Path
import json
from natsort import natsorted
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from skimage import io
from neuroprocessing.scripts.parse_csv import parse_csv, process_data


In [ ]:
def _get_sync_info(sync_csv_path):
    """Get dict of sync info (stim time, etc) from the sync csv file
    """
    df_daq = parse_csv(sync_csv_path)

    df_frames = process_data(
        df_daq,
        col_camera="camera",
        col_stim="button"
    )

    stim_onset_frame = int(df_frames.loc[df_frames["stimulated"], "frame"].iloc[0])
    df_frames.set_index("frame", inplace=True)
    # print info
    print(f"Stimulus onset frame: {stim_onset_frame}")
    print(f"Stimulus onset time (s): {df_frames.loc[stim_onset_frame, 'time']}")
    stim_duration_frames = sum(df_frames['stimulated'])
    frame_time_s = df_frames.loc[stim_onset_frame, 'frametime']
    framerate_hz = 1/frame_time_s

    sync_info = {
        "stim_onset_frame": stim_onset_frame,
        "stim_duration_frames": stim_duration_frames,
        "frame_time_s": frame_time_s,
        "framerate_hz": framerate_hz
    }
    print(f"Stimulus duration (frames): {stim_duration_frames}")
    print(f"Stimulus duration (s): {stim_duration_frames/framerate_hz}")
    print(f"Frame duration (ms): {frame_time_s*1000}")
    print(f"Framerate (Hz): %.2f" % (framerate_hz))
    return sync_info

def process_trial(trial_dir, params):
    """Process single trial
    
    Process a single imaging trial and save the processed tiff stack to the same directory as the original tiff stack.

    An imaging trial can be split into 2+ videos due to tiff file size limits.

    Stimulus is assumed to be in the first video.

    Inputs:
        trial_dir: str 
            Path to the directory containing pre-processed tiff stack
        decimation_factor: int
            Factor by which the stack has been decimated
            **TODO** can probably hardcode the decimation factor below
        sync_csv: Path or int
            If Path, path to the sync csv file. If int, the frame number of the stimulus onset.
    """

    # load sync json
    with open(Path(trial_dir).parent / "sync_info.json", "w") as f:
        sync_info = json.load(f)
    

    if type(tif_paths) == str:
        tif_paths = [tif_paths]

    # iterate through tifs and concatenate stacks
    stack_stim = []
    first_movie = True
    for tif_path in tif_paths:
        stack = io.imread(tif_path)
        CROP_PX = 20
        BOTTOM_PERCENTILE = 5
        # crop image a little bit to remove the black border
        stack = stack[:, CROP_PX:-CROP_PX, CROP_PX:-CROP_PX]

        # not processing baseline frames for now
        if first_movie:
            # if first movie, add only the frames after the stimulus onset
            stack_stim.append(stack[int((sync_info["stim_onset_frame"]+100)/sync_info['decimation_factor']):, :, :])
            first_movie = False
        else:
            # if subsequent movie, add all frames
            stack_stim.append(stack)

    stack_stim = np.concatenate(stack_stim, axis=0)
    # find bottom X% of pixels in image
    bottom10 = np.percentile(stack_stim, BOTTOM_PERCENTILE, axis=(1,2), keepdims=True).astype(np.uint16)
    # subtract bottom X% from all pixels
    stack_stim -= bottom10
    stack_stim -= stack_stim.min(axis=0, keepdims=True)
    stack_stim[stack_stim > 30000] = 0

    # save tiff stack with the name of the first tiff in the stack
    io.imsave(Path(tif_path).parent / ("processed_" + Path(tif_paths[0]).name), stack_stim)


# Run
* This processes a single trial

In [ ]:

# location where all experimental data is kept
dir_experiments = Path("/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections")

# path to a particular (or set of) experiment(s) for analysis
date = "2024-03-06"
expt = 'Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1' # 'RHL_hist' # "RHL_hist" # LHL_saline
fp_csv = natsorted((dir_experiments/date/expt).glob("*.csv"))[0]
fp_tifs = natsorted((dir_experiments/date/expt).glob("aligned*.tif"))

decimation_factor = 8
img_d = process_trial(fp_tifs, decimation_factor, sync_csv=fp_csv)


## Test out preprocessing steps (load, de-noising, and motion correction)

In [45]:
import json, tqdm
from skimage.measure import block_reduce
from neuroprocessing.align import StackAligner

def preprocess_trial(trial_dir:str, params:dict):
    """Preprocessing of a single imaging trial

    A single trial may have multiple videos due to tiff file size limits.
        1. Downsample the tiff stack
        2. Motion correction
    """
    fp_tifs = natsorted(Path(trial_dir).glob("Zyla*.tif"))
    fp_csv = natsorted(Path(trial_dir).glob("*.csv"))[0]
    sync_info = _get_sync_info(fp_csv)
    # save sync info as json
    with open(Path(trial_dir).parent / "sync_info.json", "w") as f:
        json.dump(sync_info, f)
    
    params['downsample_factor'] = 8

    stack_list = []
    for fp_tif in fp_tifs:
        stack_list.append(io.imread(fp_tif))

    # Downsample image
    stack_downsampled = block_reduce(np.concatenate(stack_list, axis=0), block_size=(params['downsample_factor'], 1, 1), func=np.mean)
    del(stack_list)

    aligner = StackAligner(
            filepath=stack_downsampled # <<NOT WORKING NOW>> change to accept stack_downsampled!
        )
    aligner.align()
    # some additional func to do the actual transformation?
    
    # save aligned stack
    stack_aligned = aligner.stack_aligned

    io.imsave(Path(trial_dir).parent / ("aligned_downsampled_" + Path(fp_tifs[0]).name), stack_aligned.astype(np.uint16))

params = {
    "downsample_factor": 8
}
preprocess_trial('/Users/ilya_arcadia/Neuroimaging_local/Processed/Injections/2024-03-06/Zyla_30min_LHL_27mMhistinj_1pt75pctISO_1',
                 params)

TypeError: preprocess_trial() missing 1 required positional argument: 'params'

In [40]:
stack_reduced

array([[[10023.5  , 10636.125, 10876.625, ..., 15226.125, 15499.875,
         15565.875],
        [10264.875, 10857.25 , 11100.375, ..., 15353.375, 15657.875,
         15666.   ],
        [10490.125, 11041.125, 11399.5  , ..., 15780.625, 15954.5  ,
         16059.875],
        ...,
        [ 8071.5  ,  8093.625,  7888.   , ...,  5683.375,  5685.375,
          5881.75 ],
        [ 8288.875,  8223.   ,  8104.875, ...,  5801.875,  5849.5  ,
          6100.375],
        [ 8222.125,  8283.5  ,  8343.375, ...,  6093.625,  6090.375,
          6204.125]],

       [[ 9936.375, 10569.125, 10759.   , ..., 15151.   , 15636.25 ,
         15733.25 ],
        [10186.75 , 10792.875, 11028.125, ..., 15467.   , 15822.   ,
         15764.875],
        [10418.375, 11061.125, 11241.125, ..., 15707.625, 15823.   ,
         16193.375],
        ...,
        [ 8168.375,  8115.75 ,  7919.375, ...,  5693.75 ,  5805.75 ,
          5883.   ],
        [ 8200.   ,  8177.5  ,  8124.375, ...,  5845.25 ,  5848.625,
   